# <font color=red>LangChain:  Demos of creating a tool to wrap use of TinyDB </font>
- https://docs.langchain.com/docs

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Read a file of reaction data and insert it into a TinyDB</br>for use in following demos

In [ ]:
# pip install tinydb

from tinydb import TinyDB, Query

db = TinyDB('reactions.json')

with open("reactions.txt") as f:
    for line in f:
        (reactID,metabolites,gibbs,chemistry) = line.strip().split("\t")   # tab-separated
        info = {"reactionID": reactID, "metabolites": metabolites, "gibbs": gibbs, "chemistry": chemistry}
        db.insert(info)

Qobj = Query()
r = db.search(Qobj.reactionID == "rxn02146")
print(r)

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 1 of user-defined tool to wrap access to TinyDB (OpenAI GPT-4)
</font></span>

In [ ]:
import os, re, time, openai, langchain

from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType

## begin tool; this could go into a module for import <--------
reactions_db_tool_description = """
Use this tool to retrieve info from the TinyDB reactions database.
The fields of the database are:  reactionID, metabolites, gibbs, chemistry.
To use this tool, you must provide a query that refers to a reactionID for
which you need information.
"""

class ReactionsDBTool(BaseTool):
    from tinydb import TinyDB, Query
    name = "ReactionsDBTool"
    description = reactions_db_tool_description
    db : TinyDB = None
    query : Query = None

    def _run(self, reactionID: str):
        from tinydb import TinyDB, Query   # again in this context
        if not self.query:
            self.query = Query()
        if not self.db:
            self.db = TinyDB("reactions.json")
        rec = self.db.search(self.query.reactionID == reactionID)
        return rec

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")
## end tool

tools = [ReactionsDBTool()]

llm = ChatOpenAI(temperature=0.0, model_name='gpt-4')

agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=False,   # True,
    max_iterations=3,
    early_stopping_method='generate',
)

response = agent("What is the gibbs energy value for reaction rxn02146 ?")
print(response["output"])

<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 2 of user-defined tool to wrap access to TinyDB (Mistral-7B)
</font></span>
Note you will need a local install of Mistral-7B-Instruct-v0.1 to run this demo.

In [ ]:
import os, re, time, langchain

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.llms import HuggingFacePipeline
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType


## begin tool; this could go into a module for import
reactions_db_tool_description = """
Use this tool to retrieve info from the TinyDB reactions database.
The fields of the database are:  reactionID, metabolites, gibbs, chemistry.
To use this tool, you must provide a query that refers to a reactionID for
which you need information.
"""

class ReactionsDBTool(BaseTool):
    from tinydb import TinyDB, Query
    name = "ReactionsDBTool"
    description = reactions_db_tool_description
    db : TinyDB = None
    query : Query = None

    def _run(self, reactionID: str):
        from tinydb import TinyDB, Query   # again in this context
        if not self.query:
            self.query = Query()
        if not self.db:
            self.db = TinyDB("reactions.json")
        rec = self.db.search(self.query.reactionID == reactionID)
        return rec

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")
## end tool

tools = [ReactionsDBTool()]

model_id = "./Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
)
model.eval()

pipe = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    # stopping_criteria=stopping_criteria,  # without this model rambles during chat
    do_sample=True,
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # without this, output may begin repeating
)

llm = HuggingFacePipeline(pipeline=pipe)

agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=False,   # True,
    max_iterations=3,
    early_stopping_method='generate',
)

query = "What is the gibbs energy value for reaction rxn02146 ?"
response = agent(query)
print("QUERY:",query)
print("REPLY:",response["output"])